MSD-I dataset (mapping, metadata, annotations and links to images)
Data splits and feature vectors for TISMIR single-label classification experiments 

In [2]:
import numpy as np
import pandas as pd
import json
import tqdm
from collections import Counter

In [ ]:
dataset = "../../../data/MSD-I_dataset.tsv"

In [ ]:
dataset_df = pd.read_csv(dataset, sep='\t')

In [ ]:
dataset_df

In [ ]:
dataset_track_ids = set(dataset_df.msd_track_id.values)

In [ ]:
dataset_track_ids

# Lyrics data

In [ ]:
lyrics_train_file = '../../../song-lyrics/data/clean/mxm_dataset.json'

In [ ]:
with open(lyrics_train_file) as json_data:
    data = json.load(json_data)

In [ ]:
len(data['songs'])

In [ ]:
data['songs']

In [ ]:
data['songs'][0]['track_id'] 

In [ ]:
lyrics_ids = []
for song in tqdm.tqdm(data['songs']):
    track_id = song['track_id'] 
    lyrics_ids.append(track_id)

In [ ]:
lyrics_ids_set = set(lyrics_ids)

In [ ]:
data['songs'][0]['track_id'] in dataset_track_ids

In [ ]:
is_in_array = []
for id_ in list(dataset_track_ids):
    is_in_array.append(id_ in lyrics_ids_set)

In [ ]:
Counter(is_in_array)

# Getting songs names

In [ ]:
song_names_file = '../../../lyrics_datasets/unique_tracks.txt'

In [ ]:
song_names_df = pd.read_csv(song_names_file, sep='<SEP>', header=None)

In [ ]:
song_names_df.columns = ['track_id', 'song_id', 'artist_name', 'song_title']

In [ ]:
song_names_df.head()

In [ ]:
len(song_names_df)

In [ ]:
song_names_dict = dict(zip(song_names_df.track_id.values, song_names_df.song_title.values))

In [ ]:
song_names_dict

In [ ]:
dataset_df.head()

In [ ]:
dataset_df['song_title'] = dataset_df['msd_track_id'].copy()

In [ ]:
dataset_df['song_title'] = dataset_df['song_title'].map(song_names_dict)

In [ ]:
dataset_df

In [ ]:
artist_dict = dict(zip(song_names_df.track_id.values, song_names_df.artist_name.values))

In [ ]:
dataset_df['artist'] = dataset_df['msd_track_id'].map(artist_dict)

In [ ]:
len(dataset_df['artist'])

# Getting lyrics

In [ ]:
lyrics_file = '../../../lyrics_datasets/songdata.csv'

In [ ]:
lyrics_df = pd.read_csv(lyrics_file)

In [ ]:
lyrics_df.head()

In [ ]:
lyrics_dict = dict(zip(lyrics_df.song.values, lyrics_df.text.values))

In [ ]:
dataset_df['song_lyrics'] = dataset_df['song_title'].map(lyrics_dict)

In [ ]:
dataset_df['song_lyrics']

In [ ]:
len(dataset_df.song_lyrics)

In [ ]:
len(dataset_df.song_lyrics.dropna())

In [ ]:
dataset_df

In [ ]:
len(dataset_df)

In [ ]:
len(dataset_df) - len(dataset_df.song_lyrics.dropna())

# Load dataset

# Parse lyrics freak

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup

In [4]:
def get_song_link_by_name(artist, song_name, website='https://www.lyricsfreak.com'):
    url = 'https://www.lyricsfreak.com/search.php?q=%s %s' % (artist, song_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    floatfix = soup.find('div', attrs={'class':"maincont floatfix"})
    list_cell = floatfix.find('div', attrs={'class':"lf-list__cell lf-list__meta"})
    if list_cell is not None:
        song_link = list_cell.find('a')['href']
        full_song_link = website + song_link
    else:
        full_song_link=None
    return full_song_link

In [ ]:
get_song_link_by_name(dataset_df.loc[3, 'artist'], dataset_df.loc[3, 'song_title'])

In [ ]:
dataset_df['lyrics_link'] = None

In [ ]:
for i in tqdm.tqdm(range(len(dataset_df))):
    dataset_df.loc[i, 'lyrics_link'] = get_song_link_by_name(dataset_df.loc[i, 'artist'], dataset_df.loc[i, 'song_title'])

In [ ]:
dataset_df.to_csv('dataset.csv', index=False)

In [ ]:
loaded = pd.read_csv('dataset.csv')

In [ ]:
loaded

# Save lyrics

In [5]:
def get_lyrics(url):
    if url is not None:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        share_text = soup.find('div', attrs={'class':"js-share-text"})
        lyrics_txt = share_text.find('div', attrs={'class':"lyrictxt js-lyrics js-share-text-content"})
        filtered_lyrics = filter(lambda x: type(x) == bs4.element.NavigableString and x.strip() != '', lyrics_txt.contents)
        text = ''
        for line in  filtered_lyrics:
            text += line.strip() + ' '
        return text.lower()
    else:
        return None

In [6]:
dataset_df = pd.read_csv('dataset_checkpoint.csv')

In [7]:
dataset_df.head()

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,song_title,song_lyrics,artist,lyrics_link,lyrics_text
0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/slee...,The Illusionist,NaN,Scar Symmetry,https://www.lyricsfreak.com/s/scar+symmetry/th...,as an outcast i paced defeat seeking comfort i...
1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/slee...,Tuolumne,NaN,Eddie Vedder,NaN,NaN
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,Weekend Driver,NaN,Foghat,https://www.lyricsfreak.com/f/foghat/weekend+d...,dave peverett - riff bros. music - ascap waiti...
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,Too Alive,NaN,The Breeders,https://www.lyricsfreak.com/b/breeders/too+ali...,i took off when will will make no way steal it...
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,Ride The Darker Wave,NaN,Sebadoh,https://www.lyricsfreak.com/s/sebadoh/ride+the...,you make your words so nice i'm waiting you to...


In [8]:
for i in tqdm.tqdm(range(len(dataset_df))):
    try:
        if dataset_df.loc[i, 'lyrics_link'] is None and dataset_df.loc[i, 'lyrics_text'] is None:
            dataset_df.loc[i, 'lyrics_link'] = get_song_link_by_name(dataset_df.loc[i, 'artist'], dataset_df.loc[i, 'song_title'])
        dataset_df.loc[i, 'lyrics_text'] = get_lyrics(dataset_df.loc[i, 'lyrics_link'])
        if i != 0 and i % 1000 == 0:
            dataset_df.to_csv('dataset_checkpoint_2.csv', index=False)
    except:
        pass

100%|██████████| 30713/30713 [34:16<00:00, 14.94it/s]   


In [9]:
dataset_df

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,song_title,song_lyrics,artist,lyrics_link,lyrics_text
0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/slee...,The Illusionist,NaN,Scar Symmetry,https://www.lyricsfreak.com/s/scar+symmetry/th...,as an outcast i paced defeat seeking comfort i...
1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/slee...,Tuolumne,NaN,Eddie Vedder,NaN,NaN
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/slee...,Weekend Driver,NaN,Foghat,https://www.lyricsfreak.com/f/foghat/weekend+d...,dave peverett - riff bros. music - ascap waiti...
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/slee...,Too Alive,NaN,The Breeders,https://www.lyricsfreak.com/b/breeders/too+ali...,i took off when will will make no way steal it...
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/slee...,Ride The Darker Wave,NaN,Sebadoh,https://www.lyricsfreak.com/s/sebadoh/ride+the...,you make your words so nice i'm waiting you to...
5,TRUSOQD128F92CFEA5,Rock,7,train,ARZHQDD1187FB5B871,http://artwork-cdn.7static.com/static/img/slee...,Carmen,Last night she ran away \nLeft a letter by th...,Lush,https://www.lyricsfreak.com/l/lush/carmen_2008...,(anderson/berenyi) carmen tells me what she's ...
6,TRKCEWE128F425BF56,Rock,9,train,ARRSEGJ1187B9A67D9,http://artwork-cdn.7static.com/static/img/slee...,Skizziert,NaN,Bosse,NaN,NaN
7,TRRCUNU128F932C084,Reggae,10,train,ARDI8UK1187FB369FB,http://artwork-cdn.7static.com/static/img/slee...,Eurose,NaN,Ohrbooten,NaN,NaN
8,TRRNTLC128F93366F6,Rock,11,train,ARTNHBW1187B9A5482,http://artwork-cdn.7static.com/static/img/slee...,Sugar Spun Sister,Her hair \nSoft drifted snow \nDeath white ...,The Stone Roses,https://www.lyricsfreak.com/s/stone+roses/suga...,her hair soft drifted snow death white i'd lik...
9,TRSEIMP12903CE2A6B,Reggae,12,train,ARP06GY1187B98B0F0,http://artwork-cdn.7static.com/static/img/slee...,Imperial Version,NaN,Yami Bolo,NaN,NaN


In [ ]:
dataset_df.isna

In [13]:
l = np.array(dataset_df.song_lyrics.isna())

In [15]:
~l

array([False, False, False, ..., False,  True, False])

In [16]:
l_ = np.array(dataset_df.lyrics_text.isna())

In [18]:
~l_

array([ True, False,  True, ..., False, False, False])

In [23]:
res = ~l | ~l_

In [24]:
np.count_nonzero(res)

10744

In [ ]:
print ('f')

In [ ]:
for i in tqdm.tqdm(range(len(dataset_df))):
    #dataset_df.loc[i, 'lyrics_link'] = get_song_link_by_name(dataset_df.loc[i, 'artist'], dataset_df.loc[i, 'song_title'])

In [ ]:
dataset_df